In [17]:
import pandas as pd

In [27]:
df = pd.read_csv("./data/sample_companies.csv")

df.head()

,company,ticker,sector,tech_focus
0,Guardant Health,NASDAQ:GH,HealthTech,AI-powered blood tests for early cancer detection
1,C3.ai,NYSE:AI,Enterprise AI,Enterprise AI platform with sector-specific mo...
2,CrowdStrike,NASDAQ:CRWD,Cybersecurity,AI-powered endpoint protection and threat inte...
3,Darktrace,LSE:DARK,Cybersecurity,Self-learning AI for enterprise threat detection
4,Duolingo,NASDAQ:DUOL,EdTech,AI and gamification to personalize language le...


In [28]:
df['ticker'] = df['ticker'].apply(lambda x : x.split(":")[1])

In [29]:
df.head()

,company,ticker,sector,tech_focus
0,Guardant Health,GH,HealthTech,AI-powered blood tests for early cancer detection
1,C3.ai,AI,Enterprise AI,Enterprise AI platform with sector-specific mo...
2,CrowdStrike,CRWD,Cybersecurity,AI-powered endpoint protection and threat inte...
3,Darktrace,DARK,Cybersecurity,Self-learning AI for enterprise threat detection
4,Duolingo,DUOL,EdTech,AI and gamification to personalize language le...


In [30]:
#import openai, azure.search.documents as s
from yfinance import Ticker
import time 
#from azure.core.credentials import AzureKeyCredential

#openai.api_key = "AZURE_OPENAI_KEY"
#index_name = "companies"
#client = s.SearchIndexClient(endpoint, cred)

#def embed(txt):  # one-liner wrapper
#    return openai.Embedding.create(engine="text-embedding-3-small", input=txt)["data"][0]["embedding"]

def build_doc(ticker, df_row):
    yf = Ticker(ticker)
    info = yf.info
    about = info.get("longBusinessSummary", "")
    sub_sector = row['sector']
    #prod  = " ".join(about.split(".")[:2])  # crude product snippet

    return {
        "ticker": ticker,
        "name": info["shortName"],
        "sector": info.get("sector", "Unknown"),
        "sub_sector": sub_sector,
        "country": info.get("country", "Unknown"),
        "ebitda_musd": round(info.get("ebitda", 0)/1e6, 1),
        "rev_growth_pct": round(info.get("revenueGrowth", 0), 2),
        "market_cap_musd": round(info.get("marketCap", 0)/1e6, 1),
        #"share_price": yf.history("1d")["Close"][-1],
        #"ai_flag": "ai" in about.lower(),
        "long_description": about,
        #"long_descriptionVector": embed(about),
        #"product_summary": prod,
        #"product_summaryVector": embed(prod),
        "@search.action": "mergeOrUpload"
    }

docs = []

for _, row in df.iterrows():
    ticker = row['ticker']
    sub_field = row['sector']
    
    try:
        doc = build_doc(ticker)
        docs.append(doc)
        
    except Exception as e:
        print(f"Error processing {ticker}: {e}")


In [ ]:
tickers = ["GH", "AI","CRWD","DARK-USD","DUOL", "STEM", "PLTR"]
#"AI","CRWD","DARK","DUOL", "STEM", "PLTR", "PATH", "VCYT", "ENPH", "AFRM", "VEEV", "ISRG", " BFLY", "TDOC", "IQV", "SYK","SHL.DE", "SNOW","DDOG","ESTC","AYX","NOW","CRM","NET","S","FTNT","CHPT","SHLS","WDAY","PATH"]
docs = []   
for t in tickers:
    try:
        docs.append(build_doc(t))
        time.sleep(1)
    except:
        print(f"Could not processticker: {t}")
        pass
    
#docs = [build_doc("MSFT")]
#uploader = s.SearchClient(endpoint, index_name, cred)
#uploader.upload_documents(docs)

In [31]:
from pprint import pprint

In [32]:
for doc in docs:
    pprint(doc) 

{'@search.action': 'mergeOrUpload',
 'country': 'United States',
 'ebitda_musd': -412.7,
 'long_description': 'Guardant Health, Inc., a precision oncology company, '
                     'provides blood and tissue tests, and data sets in the '
                     'United States and internationally. The company provides '
                     'precision oncology testing services comprising '
                     'Guardant360, a panel of genetic tests; Guardant360 LDT '
                     'that measures 730+ genes and supports all '
                     'guideline-recommended biomarkers; Guardant360 CDx Test, '
                     'a liquid biopsy test; Guardant360 Response Test, a '
                     'blood-only liquid biopsy; Guardant360 TissueNext Test, a '
                     'tissue-based test with AI-powered PD-L1 detection; '
                     'GuardantINFINITY Test that provides insights into the '
                     'complexities of tumor molecular profiles and immu

In [9]:
docs[0]

{'ticker': 'MSFT',
 'name': 'Microsoft Corporation',
 'sector': 'Technology',
 'country': 'United States',
 'ebitda_musd': 149172.994048,
 'rev_growth_pct': 13.3,
 'market_cap_musd': 3567359.09888,
 'share_price': np.float64(479.9649963378906),
 'ai_flag': True,
 'long_description': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power apps, and power automate; and on-premises ERP and CRM applications. The Intelligent Cloud segment offers server products and cloud services, 

In [7]:
pprint(docs[0]['long_description'])

('Microsoft Corporation develops and supports software, services, devices and '
 'solutions worldwide. The Productivity and Business Processes segment offers '
 'office, exchange, SharePoint, Microsoft Teams, office 365 Security and '
 'Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer '
 'services, such as Microsoft 365 consumer subscriptions, Office licensed '
 'on-premises, and other office services. This segment also provides LinkedIn; '
 'and dynamics business solutions, including Dynamics 365, a set of '
 'intelligent, cloud-based applications across ERP, CRM, power apps, and power '
 'automate; and on-premises ERP and CRM applications. The Intelligent Cloud '
 'segment offers server products and cloud services, such as azure and other '
 'cloud services; SQL and windows server, visual studio, system center, and '
 'related client access licenses, as well as nuance and GitHub; and enterprise '
 'services including enterprise support services, industry sol